※開発環境はGoogle Colabです。

**0.概要**

東証プライムの企業のPBRの推移データを計算してグラフで表示させたい。

*   PBR =　時価総額（株価 * 発行数） / 株主資本

計算をするために、まずは必要なデータを取得する。

プライムの全銘柄のティッカーを取得。
さらに、バランスシートのデータ（過去２～４年分）を取得。
バランスシートは、企業が決算時に公開している。（つまり１年につき１～数回程度公開）

取得したデータは、すぐに取り出せるようにスプレットシートに保存してデータベース化。（毎回サーバーから全銘柄のデータを一括取得すると時間がかかる）

※発行数・株主資本はバランスシートのデータに含まれている。

※株価データについては、データベース化せずに、PBRを計算する際に各銘柄個別で取得する。バランスシートは年１～数個程度しかデータがないが、株価は毎日データがあり、量が多いのでデータベース化しない。
計算する際に個別で取得すればさほど時間もかからない。

**1.必要なライブラリのインポート等（環境設定）**

In [3]:
# yahooqueryから金融指標を取得できる
!pip install yahooquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.7 MB/s eta 0:00:00


In [4]:
# yahooqueryのライブラリを読み込む
from yahooquery import Ticker

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Google認証
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

**2.東証プライム全銘柄のティッカー(4桁の数字)をスプレットシートに記録**

APIで金融指標データを取得するときに、4桁のティッカーが必要になる。

yahooqueryのAPIで日本株のデータを取得するときには、ティッカーの末尾に".T"を付ける必要があるので、全銘柄の末尾に".T"を付けてシートに記入。


In [6]:
#spreadsheetを開く（東証上場銘柄一覧）
url = "https://docs.google.com/spreadsheets/d/1SxMC3GUKHCHCPUX4NTuhU6rDeVrmR3aIMqA-hNVJbIc/edit#gid=1407125800"
ss = gc.open_by_url(url)

worksheet0 = ss.get_worksheet(0) #一枚目のシート取得
all_values = worksheet0.get_all_values() # 一枚目のシートの全データを取得
df_alltokyo = pd.DataFrame.from_records(all_values) # pandasで読み込み

In [7]:
# プライム銘柄のみを抽出(ETFやリートは不要)（データが多いので、ひとまずスタンダード、グロースはスルー）
df_prime = df_alltokyo[df_alltokyo[3].isin(["プライム（内国株式）"])]
# ティッカーと銘柄名のみ抽出
df_prime = df_prime.iloc[:, [1, 2]]
# 列名をつける
df_prime.columns = ['Ticker', 'Name']
# Tickerの末尾に".T"をつける
df_prime.insert(loc=1, column='Ticker.T', value=df_prime.iloc[:, 0].astype(str) + '.T')

df_prime

,Ticker,Ticker.T,Name
1,1301,1301.T,極洋
17,1332,1332.T,ニッスイ
18,1333,1333.T,マルハニチロ
34,1375,1375.T,雪国まいたけ
36,1377,1377.T,サカタのタネ
...,...,...,...
4246,9989,9989.T,サンドラッグ
4247,9990,9990.T,サックスバー ホールディングス
4248,9991,9991.T,ジェコス
4251,9995,9995.T,グローセル


**3.各銘柄のBS(バランスシート)のデータ（過去２～４年）を取得してシートに記入**

In [ ]:
# データフレームをGoogleスプレッドシートに書き出す
!pip install gspread_dataframe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# gspread_dataframeのライブラリをインポート
from gspread_dataframe import set_with_dataframe

In [ ]:
ss.add_worksheet(title="Sheet2", rows=1834, cols=3)

<Worksheet 'Sheet2' id:1011082154>

In [ ]:
# df_primeをワークシートに書き出し
set_with_dataframe(ss.worksheet("Sheet2"), df_prime, include_index=True)

In [ ]:
# 全銘柄のB/S情報を取得
bs = Ticker(df_prime["Ticker.T"]).balance_sheet()

In [ ]:
type(bs)

pandas.core.frame.DataFrame

In [ ]:
# 新しいシート追加
ss.add_worksheet(title="Sheet3_BS", rows=5865, cols=129)
# bsをシートに書き出し
set_with_dataframe(ss.worksheet("Sheet3_BS"), bs, include_index=True)